Business Question Brainstorm:
- Cost of Plane
- Personal vs. Commercial
- Spatial:  
           - United States:
                    - Can use lat/long to look at geography 
           - International vs US
           - Developed vs. Developing (and temporal?)


Data Cleaning List:
- Subset by time period
- Subset into developed vs. developing - before and after developing, then developed
- Remove dupes based on Event.id

## Import Packages and Read in Data

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pandas.api.types as ptypes

#pd.set_option("max_rows", None)

In [7]:
df = pd.read_csv("data/AviationData.csv", delimiter=",")
#finalProjectPhase01/Avation_Safety_Data_Analysis/data/AviationData.csv

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

## Initial Data Exploration

In [ ]:
df.head()

In [ ]:
df.info()

## Convert Event.Date to Date Type

In [ ]:
# Coerce 'Event.Date' into pandas date time object
df["Event.Date"] = pd.to_datetime(df['Event.Date'], format='%Y-%m-%d', errors = 'coerce')
# Assert that Event.Date is not a datetime64 dtype:
assert ptypes.is_datetime64_any_dtype(df['Event.Date'])



## Clean Event.Id Column
### This removes nulls from Event.Id, Investigation.Type, Accident.Number, and Event.Date

In [ ]:
# Drop rows where 'Event.Id' is NULL
df_clean = df.dropna(subset=['Event.Id'])


In [ ]:
# Drop duplicates using the Event.Id column
df_clean1 = df_clean.drop_duplicates(subset=['Event.Id'])


In [ ]:
# After dropping rows where NOT Event.Id null,
# We now have a dataset where first 4 columns have no nulls
# We think this makes sense because without an id or accident number, the event wasn't properly documented
df_clean.isna().sum()

# Subset based on Event.Date
- We wanted to only look at past 30 years because planes older than that likely no longer flying

In [ ]:
# Subset data to look at past 30 years of data:
df_30 = df_clean[df_clean['Event.Date'] >= '1993-01-01']

In [ ]:
# Check Date Range
print(df_30['Event.Date'].min())
print(df_30['Event.Date'].max())


# Create Total Injuries Column

In [ ]:
# Create Total Injuries Column
df_30['Total.Injuries'] = df_30['Total.Fatal.Injuries'] + df_30['Total.Minor.Injuries'] + df_30['Total.Serious.Injuries'] 
# Sanity Check new 'Total.Injuries' Column:
df_30[['Total.Injuries', 'Total.Fatal.Injuries', 'Total.Minor.Injuries', 'Total.Serious.Injuries']].head()

# Clean Make and Model Columns

In [ ]:
#Cleaning the Make column (this gets rid of a lot of unique values)
#Start by making everything lowercase
df_clean1['Make'] = df_clean1['Make'].str.lower()
#capitalize the beginning of each word
df_clean1['Make'] = df_clean1['Make'].str.title()
#Get rid of whitespace
df_clean1['Make'] = df_clean1['Make'].str.strip()

In [ ]:
#Cleaning the Model column (I don’t think this got rid of any unique values)
#Start by making everything uppercase
df_clean1['Model'] = df_clean1['Model'].str.upper()
#Get rid of whitespace
df_clean1['Model'] = df_clean1['Model'].str.strip()

In [ ]:
df_clean1['Model']

# Clean Aircraft.Category Column:
- Here we will throw out only those Aircraft.Category rows where Airplane or NULL
- Concatenate Make and Model = Make_Model
- Keep the rows that are (Aircraft.Category = NULL) ONLY IF...
- The Make_Model value is one that exists in rows that are (Aircraft.Category = 'Airplane')

In [ ]:
#Get rid of non airplane rows but keep the nulls
#First fill nulls with ‘n/a’
df_clean1['Aircraft.Category'].fillna('n/a', inplace=True)
#Keep only rows with ‘Airplane’ or ‘n/a’
df_clean2 = df_clean1.loc[(df_clean1['Aircraft.Category'] == 'Airplane') | (df_clean1['Aircraft.Category'] == 'n/a')]


In [ ]:
df_clean2['Aircraft.Category'].value_counts()

In [ ]:
# Create concatenated Make.Model column:
df_clean2["Make.Model"] = df_clean2['Make'].astype(str) +"_"+ df_clean2["Model"]

In [ ]:
df_clean2["Make.Model"].head()

In [ ]:
# Preview counts for each Make.Model, grouped on Aircraft.Category:
df_clean2.groupby(['Aircraft.Category','Make.Model']).size()


### Split into two dataframes:

In [ ]:
# Isolate a dataframe only containing rows where category is Airplane:
df_plane = df_clean2[df_clean2['Aircraft.Category'] == 'Airplane']
# And one for category = n/a:
df_na = df_clean2[df_clean2['Aircraft.Category'] == 'n/a']

### Explore results:

In [ ]:
# Category = Airplane

# 27,520 rows
# 7,484 unique Make.Model values
print(df_plane.shape)
print(df_plane['Make.Model'].unique().shape)
df_plane['Make.Model'].tail()

In [ ]:
# Category = n/a

# 55,770 rows
# 11,038 unique Make.Model values
print(df_na.shape)
print(df_na['Make.Model'].unique().shape)
df_na['Make.Model'].tail()

### Filter the **category = n/a** rows to only contain the Make.Model values of Airplanes

In [ ]:
# Get a list of all of the unique values of "Make.Model" where aircraft.category = 'Aiplane':
airplane_make_model_list = list(df_plane['Make.Model'].unique())

In [ ]:
# Go back to original dataframe "df_clean2" before we split it, and filter to only Make.Model values in airplane_make_model_list
df_ap_mm = df_clean2[df_clean2['Make.Model'].isin(airplane_make_model_list)]

In [ ]:
# Sanity Check:
# Number of rows where 'Aircraft.Category' = 'Airplane' vs. Total Rows
print(df_ap_mm['Aircraft.Category'].value_counts())
print(df_ap_mm.shape)

### Now we only have Make.Model rows that we know are Airplanes, so we can replace all 'n/a' values with 'Airplane'

In [ ]:
# Fill 'n/a' value with 'Airplane':
df_ap_mm['Aircraft.Category'] = df_ap_mm['Aircraft.Category'].replace(['n/a'], 'Airplane')
